In [1]:
import numpy as np
import pandas as pd
from scipy import optimize
from scipy.stats import chi2
import io

# 1. Load data
print("Loading Planck CMB power spectrum data...")
filename = 'https://irsa.ipac.caltech.edu/data/Planck/release_3/ancillary-data/cosmoparams/COM_PowerSpect_CMB-TT-full_R3.01.txt'
data = pd.read_csv(filename, sep='\s+', comment='#',
                  names=['l', 'Dl', 'error_low', 'error_high', 'BestFit'])

print("First rows of loaded data:")
print(data.head())

# Use all available data
valid_data = data.copy()
max_l = valid_data['l'].max()
print(f"\nAnalysis covers the full range: l from {valid_data['l'].min()} to {valid_data['l'].max()}")
print(f"Total data points: {len(valid_data)}")

# 2. Define physically meaningful l ranges for analysis
l_ranges = [
    (2, 10, "Largest scales (quadrupole/octupole)"),
    (11, 40, "Sachs-Wolfe plateau"),
    (41, 150, "Equality hump"),
    (151, 350, "First acoustic peak"),
    (351, 650, "Second acoustic peak"),
    (651, 950, "Third acoustic peak"),
    (951, 1500, "Higher acoustic peaks"),
    (1501, 2500, "Silk damping"),
    (2, max_l, "Full range")
]

# 3. CMB power spectrum models

def standard_lcdm_power_spectrum(l, amplitude_scale=1.0):
    """
    Standard ΛCDM model for power spectrum
    """
    # Sachs-Wolfe plateau with gradual transition to acoustic peaks
    plateau_level = 1000.0  
    
    # Small l - plateau with slight tilt
    Dl_low = plateau_level * (l/10)**(-0.15) * amplitude_scale
    
    # Intermediate l - approximation of acoustic peaks
    Dl_mid = plateau_level * (l/200)**(0.2) * (1 + 0.5*np.sin(np.pi*l/200)) * amplitude_scale
    
    # Large l - Silk damping
    Dl_high = plateau_level * (l/1000)**(-0.3) * np.exp(-(l/2000)**2) * amplitude_scale
    
    # Smooth transition between regimes
    weight_low = 1.0 / (1.0 + (l/30)**2)
    weight_mid = 1.0 / (1.0 + (l/500)**2)
    
    Dl = weight_low * Dl_low + (1 - weight_low) * (weight_mid * Dl_mid + (1 - weight_mid) * Dl_high)
    
    return Dl

def dimension_flow_model(l, amplitude_scale=1.0, D0=2.7, l0=10.0, alpha=1.0):
    """
    Power spectrum model incorporating varying effective dimension
    
    Parameters:
    - l: multipole moments
    - amplitude_scale: overall amplitude scaling
    - D0: effective dimension at largest scales (small l)
    - l0: characteristic scale for transition
    - alpha: transition shape parameter (controls steepness)
    """
    # Get standard ΛCDM spectrum
    standard_spectrum = standard_lcdm_power_spectrum(l, amplitude_scale=1.0)
    
    # Effective dimension as function of multipole
    # For l << l0: D_l ≈ D0
    # For l >> l0: D_l ≈ 3 (standard three-dimensional space)
    D_l = 3.0 - (3.0 - D0) / (1.0 + (l/l0)**alpha)
    
    # Modification factor based on dimension difference from 3
    beta_l = 0.5 * np.exp(-l/20)  # β(l) increases with decreasing l
    modification = 1.0 - beta_l * np.abs(3.0 - D_l)
    
    # Apply modification and amplitude scaling
    modified_spectrum = standard_spectrum * modification * amplitude_scale
    
    return modified_spectrum, D_l

# 4. Calculate statistical measures
def calculate_statistics(data, model_func, parameters, n_params):
    """
    Calculates χ², reduced χ², BIC and AIC for a given model and parameters
    """
    l = data['l'].values
    Dl = data['Dl'].values
    errors = (data['error_high'].values + data['error_low'].values) / 2
    n_data = len(l)
    
    # Calculate model predictions
    if model_func.__name__ == 'dimension_flow_model':
        Dl_model, _ = model_func(l, *parameters)
    else:
        Dl_model = model_func(l, *parameters)
    
    # Calculate χ²
    residuals = Dl - Dl_model
    chi2_value = np.sum((residuals / errors)**2)
    
    # Degrees of freedom
    dof = n_data - n_params
    
    # Information criteria
    bic = chi2_value + n_params * np.log(n_data)
    aic = chi2_value + 2 * n_params
    
    return {
        'chi2': chi2_value,
        'dof': dof,
        'reduced_chi2': chi2_value / dof if dof > 0 else np.nan,
        'bic': bic,
        'aic': aic,
        'residuals': residuals,
        'predictions': Dl_model
    }

# 5. Analyze models in different l ranges
def analyze_models_in_range(data, l_min, l_max, range_name):
    """
    Analyzes both ΛCDM and dimensional flow models in a given l range
    """
    range_data = data[(data['l'] >= l_min) & (data['l'] <= l_max)].copy()
    
    if len(range_data) == 0:
        print(f"No data in range l={l_min}-{l_max}")
        return None
    
    l = range_data['l'].values
    Dl = range_data['Dl'].values
    errors = (range_data['error_high'].values + range_data['error_low'].values) / 2
    
    # Objective function for optimization
    def objective_function(params, model_func):
        if model_func.__name__ == 'dimension_flow_model':
            model_pred, _ = model_func(l, *params)
        else:
            model_pred = model_func(l, *params)
        return np.sum(((Dl - model_pred) / errors)**2)
    
    # Analyze ΛCDM model
    lcdm_result = optimize.minimize(
        lambda params: objective_function(params, standard_lcdm_power_spectrum),
        [1.0],  # Initial amplitude_scale
        method='Nelder-Mead'
    )
    
    amplitude_scale_lcdm = lcdm_result.x[0]
    lcdm_stats = calculate_statistics(range_data, standard_lcdm_power_spectrum, [amplitude_scale_lcdm], 1)
    
    # Analyze dimensional flow model
    # Adaptive initial parameters for different ranges
    if l_min < 30:
        initial_params = [1.0, 1.5, 10.0, 1.0]  # Low initial dimension for largest scales
        bounds = [(0.1, 10.0), (0.01, 2.5), (1.0, 50.0), (0.5, 2.0)]
    elif l_min < 150:
        initial_params = [1.0, 2.0, 30.0, 1.0]  # Medium dimension for intermediate scales
        bounds = [(0.1, 10.0), (1.0, 3.0), (5.0, 100.0), (0.5, 2.0)]
    elif l_min < 500:
        initial_params = [1.0, 2.5, 100.0, 1.0]  # Higher dimension for acoustic peaks
        bounds = [(0.1, 10.0), (1.5, 3.0), (50.0, 500.0), (0.5, 2.0)]
    else:
        initial_params = [1.0, 3.0, 500.0, 1.0]  # Standard 3D dimension for small scales
        bounds = [(0.1, 10.0), (2.5, 3.5), (100.0, 1000.0), (0.5, 2.0)]
    
    dim_result = optimize.minimize(
        lambda params: objective_function(params, dimension_flow_model),
        initial_params,  # [amplitude_scale, D0, l0, alpha]
        method='L-BFGS-B',
        bounds=bounds
    )
    
    amplitude_scale_dim, D0, l0, alpha = dim_result.x
    
    # Get D_l for each l in range
    _, D_l_values = dimension_flow_model(l, amplitude_scale_dim, D0, l0, alpha)
    
    # Average effective dimension in range
    D_avg = np.mean(D_l_values)
    
    # Calculate D for characteristic multipoles
    if l_min <= 2 <= l_max:
        D_2 = 3.0 - (3.0 - D0) / (1.0 + (2/l0)**alpha)  # Quadrupole
    else:
        D_2 = None
        
    if l_min <= 10 <= l_max:
        D_10 = 3.0 - (3.0 - D0) / (1.0 + (10/l0)**alpha)  # l=10
    else:
        D_10 = None
        
    if l_min <= 100 <= l_max:
        D_100 = 3.0 - (3.0 - D0) / (1.0 + (100/l0)**alpha)  # l=100
    else:
        D_100 = None
    
    if l_min <= 500 <= l_max:
        D_500 = 3.0 - (3.0 - D0) / (1.0 + (500/l0)**alpha)  # l=500
    else:
        D_500 = None
    
    dim_stats = calculate_statistics(range_data, dimension_flow_model, [amplitude_scale_dim, D0, l0, alpha], 4)
    
    # Likelihood ratio test
    delta_chi2 = lcdm_stats['chi2'] - dim_stats['chi2']
    p_value = 1 - chi2.cdf(delta_chi2, 3) if delta_chi2 > 0 else 1.0
    
    # Complete results
    results = {
        'lcdm': {
            'parameters': {'amplitude_scale': amplitude_scale_lcdm},
            'statistics': lcdm_stats
        },
        'dim_flow': {
            'parameters': {
                'amplitude_scale': amplitude_scale_dim,
                'D0': D0,
                'l0': l0,
                'alpha': alpha
            },
            'statistics': dim_stats,
            'D_avg': D_avg,
            'D_values': {
                'D_2': D_2,
                'D_10': D_10,
                'D_100': D_100,
                'D_500': D_500
            }
        },
        'comparison': {
            'delta_chi2': delta_chi2,
            'p_value': p_value,
            'delta_bic': lcdm_stats['bic'] - dim_stats['bic'],
            'delta_aic': lcdm_stats['aic'] - dim_stats['aic']
        },
        'range': {
            'l_min': l_min,
            'l_max': l_max,
            'name': range_name,
            'n_points': len(range_data)
        },
        'data': range_data
    }
    
    return results

# 6. Run analysis for all ranges
all_results = {}
for l_min, l_max, range_name in l_ranges:
    results = analyze_models_in_range(valid_data, l_min, l_max, range_name)
    if results:
        all_results[range_name] = results

# 7. Output results for all ranges
print("\n\n" + "="*60)
print("SUMMARY OF CMB POWER SPECTRUM ANALYSIS")
print("="*60)

# Create DataFrame for results table
results_data = []
for range_name, results in all_results.items():
    D0 = results['dim_flow']['parameters']['D0']
    l0 = results['dim_flow']['parameters']['l0']
    alpha = results['dim_flow']['parameters']['alpha']
    D_avg = results['dim_flow']['D_avg']
    
    chi2_lcdm = results['lcdm']['statistics']['chi2']
    chi2_dim = results['dim_flow']['statistics']['chi2']
    dof_lcdm = results['lcdm']['statistics']['dof']
    dof_dim = results['dim_flow']['statistics']['dof']
    
    reduced_chi2_lcdm = results['lcdm']['statistics']['reduced_chi2']
    reduced_chi2_dim = results['dim_flow']['statistics']['reduced_chi2']
    
    delta_chi2 = results['comparison']['delta_chi2']
    p_value = results['comparison']['p_value']
    delta_aic = results['comparison']['delta_aic']
    delta_bic = results['comparison']['delta_bic']
    
    l_range = f"{results['range']['l_min']}-{results['range']['l_max']}"
    n_points = results['range']['n_points']
    
    # Add significance markers
    significance = ""
    if p_value < 0.001:
        significance = "***"
    elif p_value < 0.01:
        significance = "**"
    elif p_value < 0.05:
        significance = "*"
    
    results_data.append({
        'Range': range_name,
        'l': l_range,
        'N': n_points,
        'D0': f"{D0:.3f}",
        'l0': f"{l0:.1f}",
        'α': f"{alpha:.2f}",
        'D_avg': f"{D_avg:.3f}",
        'χ²/dof (ΛCDM)': f"{chi2_lcdm:.1f}/{dof_lcdm}",
        'red.χ² (ΛCDM)': f"{reduced_chi2_lcdm:.3f}",
        'χ²/dof (Dim)': f"{chi2_dim:.1f}/{dof_dim}",
        'red.χ² (Dim)': f"{reduced_chi2_dim:.3f}",
        'Δχ²': f"{delta_chi2:.1f}{significance}",
        'p-value': f"{p_value:.4f}",
        'ΔAIC': f"{delta_aic:.1f}",
        'ΔBIC': f"{delta_bic:.1f}"
    })

# Create DataFrame and set formatting
results_df = pd.DataFrame(results_data)

# Sort results by l range in ascending order
# (use custom function to sort by lower bound of range)
def get_lower_bound(row):
    return int(row['l'].split('-')[0])

results_df['sort_key'] = results_df.apply(get_lower_bound, axis=1)
results_df = results_df.sort_values('sort_key')
results_df = results_df.drop('sort_key', axis=1)

# Move "Full range" to the bottom
if "Full range" in results_df['Range'].values:
    full_range_row = results_df[results_df['Range'] == "Full range"]
    other_rows = results_df[results_df['Range'] != "Full range"]
    results_df = pd.concat([other_rows, full_range_row]).reset_index(drop=True)

print("\nResults table by scale range:")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)
print(results_df.to_string(index=False))
print("\n* p < 0.05, ** p < 0.01, *** p < 0.001 (statistical significance)")

# 8. Detailed comparison for full range
print("\n\n" + "="*50)
print("DETAILED MODEL COMPARISON FOR FULL RANGE")
print("="*50)

full_range_results = all_results["Full range"]

# Create comparison table
comparison_data = []

# ΛCDM data
comparison_data.append({
    'Model': 'ΛCDM',
    'Parameters': f"amplitude_scale = {full_range_results['lcdm']['parameters']['amplitude_scale']:.4f}",
    'χ²': f"{full_range_results['lcdm']['statistics']['chi2']:.2f}",
    'dof': full_range_results['lcdm']['statistics']['dof'],
    'χ²/dof': f"{full_range_results['lcdm']['statistics']['reduced_chi2']:.4f}",
    'AIC': f"{full_range_results['lcdm']['statistics']['aic']:.2f}",
    'BIC': f"{full_range_results['lcdm']['statistics']['bic']:.2f}"
})

# Dimensional flow model data
dim_params = (f"amplitude_scale = {full_range_results['dim_flow']['parameters']['amplitude_scale']:.4f}, "
              f"D0 = {full_range_results['dim_flow']['parameters']['D0']:.4f}, "
              f"l0 = {full_range_results['dim_flow']['parameters']['l0']:.2f}, "
              f"α = {full_range_results['dim_flow']['parameters']['alpha']:.4f}")

comparison_data.append({
    'Model': 'Dimensional Flow',
    'Parameters': dim_params,
    'χ²': f"{full_range_results['dim_flow']['statistics']['chi2']:.2f}",
    'dof': full_range_results['dim_flow']['statistics']['dof'],
    'χ²/dof': f"{full_range_results['dim_flow']['statistics']['reduced_chi2']:.4f}",
    'AIC': f"{full_range_results['dim_flow']['statistics']['aic']:.2f}",
    'BIC': f"{full_range_results['dim_flow']['statistics']['bic']:.2f}"
})

# Add difference record
p_value = full_range_results['comparison']['p_value']
significance = ""
if p_value < 0.001:
    significance = "***"
elif p_value < 0.01:
    significance = "**"
elif p_value < 0.05:
    significance = "*"

comparison_data.append({
    'Model': 'Difference',
    'Parameters': "",
    'χ²': f"{full_range_results['comparison']['delta_chi2']:.2f}{significance}",
    'dof': 3,  # Difference in number of parameters
    'χ²/dof': "",
    'AIC': f"{full_range_results['comparison']['delta_aic']:.2f}",
    'BIC': f"{full_range_results['comparison']['delta_bic']:.2f}"
})

# Create and output DataFrame
comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))
print(f"\np-value = {full_range_results['comparison']['p_value']:.6f}")
print("\n* p < 0.05, ** p < 0.01, *** p < 0.001 (statistical significance)")

# 9. Calculate characteristic scales for dimension change

# Function to convert multipole l to angular scale in degrees
def l_to_angular_scale(l):
    return 180.0 / l if l > 0 else float('inf')

print("\n\n" + "="*60)
print("CHARACTERISTIC SCALES OF EFFECTIVE DIMENSION CHANGE")
print("="*60)

scales_data = []

for range_name, results in all_results.items():
    if range_name == "Full range":
        continue  # Skip full range for clearer presentation of individual scales
    
    D0 = results['dim_flow']['parameters']['D0']
    l0 = results['dim_flow']['parameters']['l0']
    alpha = results['dim_flow']['parameters']['alpha']
    
    # Calculate angular transition scale
    angular_scale = l_to_angular_scale(l0)
    
    # Calculate distance in Mpc for this angular scale
    # Approximate for flat ΛCDM cosmology and z_LS = 1100 (last scattering surface)
    angular_dist_LS = 14000  # Mpc, approximate angular distance to last scattering surface
    linear_scale = angular_scale * (np.pi/180) * angular_dist_LS  # Mpc
    
    # Characteristic transition time: age of universe - distance/c
    H0 = 67.4  # km/s/Mpc
    age_universe = 13.8  # Gyr
    transition_time = age_universe - linear_scale / 3000  # Gyr (c ≈ 0.3 Mpc/million years)
    
    scales_data.append({
        'Range': range_name,
        'l': f"{results['range']['l_min']}-{results['range']['l_max']}",
        'D0': f"{D0:.3f}",
        'l0': f"{l0:.2f}",
        'α': f"{alpha:.2f}",
        'Angular scale (°)': f"{angular_scale:.3f}",
        'Linear scale (Mpc)': f"{linear_scale:.0f}",
        'Transition time (Gyr)': f"{transition_time:.2f}" if transition_time > 0 else "< 0"
    })

# Create and output DataFrame
scales_df = pd.DataFrame(scales_data)
print(scales_df.to_string(index=False))

print("\n\nAnalysis complete.")

Loading Planck CMB power spectrum data...
First rows of loaded data:
     l        Dl  error_low  error_high  BestFit
0  2.0   225.895    132.369     533.062      NaN
1  3.0   936.920    450.471    1212.308      NaN
2  4.0   692.238    294.111     666.469      NaN
3  5.0  1501.705    574.432    1155.804      NaN
4  6.0   557.611    201.242     375.763      NaN

Analysis covers the full range: l from 2.0 to 2508.0
Total data points: 2507


SUMMARY OF CMB POWER SPECTRUM ANALYSIS

Results table by scale range:
                               Range         l    N    D0    l0    α D_avg χ²/dof (ΛCDM) red.χ² (ΛCDM) χ²/dof (Dim) red.χ² (Dim)     Δχ² p-value ΔAIC  ΔBIC
Largest scales (quadrupole/octupole)      2-10    9 0.010   2.1 2.00 2.480         5.5/8         0.682        2.4/5        0.478     3.1  0.3818 -2.9  -3.5
                 Sachs-Wolfe plateau     11-40   30 0.010  50.0 0.53 1.220       42.6/29         1.468      25.1/26        0.966 17.5***  0.0006 11.5   7.3
                   